# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install ..

Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.
Processing /Users/bytedance/Developer/todd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for todd-ai: filename=todd_ai-0.4.0-py3-none-any.whl size=108366 sha256=2fec75da0858253d8a6f2ac57730117869162ec01295bebb7baa551526073a5c
  Stored in directory: /private/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/pip-ephem-wheel-cache-_biha_6c/wheels/15/ef/5a/9fc12e257ce5cef16b333a2ed6c992ff9cbcc9167f7199e6ac
Successfully built todd-ai

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pathlib
import tempfile
from pprint import pprint
from typing import Any, NoReturn, TypedDict, cast

import todd
import torch
import torch.nn.functional as F
import torch.utils.data

Memo = dict[str, Any]

/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
[2023-08-03 16:16:32,183 45971:140704487245376][patches.py:14 todd <module>] INFO: `ipdb` is installed. Using it for debugging.


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register()
class RunnerModel(todd.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

### Runners

In [7]:
class RunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        y: torch.Tensor = self._strategy.model(batch['x'])
        loss = F.l1_loss(y, batch['y'])
        memo['loss'] = loss
        if 'log' in memo:
            memo['log']['loss'] = f'{loss.item():.3f}'
        return memo

In [8]:
class TrainerMixin(RunnerMixin):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        memo = super()._run_iter(batch, memo)
        if 'log' in memo:
            model = cast(RunnerModel, self._strategy.module)
            memo['log']['weight'] = f'{model.weight.item():.3f}'
            memo['log']['batch'] = str(batch)
        return memo

In [9]:
@todd.RunnerRegistry.register()
class CustomValidator(RunnerMixin, todd.runners.Validator):
    pass

In [10]:
@todd.RunnerRegistry.register()
class CustomIterBasedTrainer(TrainerMixin, todd.runners.IterBasedTrainer):
    pass

In [11]:
@todd.RunnerRegistry.register()
class CustomEpochBasedTrainer(TrainerMixin, todd.runners.EpochBasedTrainer):
    pass

## Validators

In [12]:
validator_demo = todd.Config(
    type='CustomValidator',
    name='custom_validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
)

In [13]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-08-03 16:16:32,830 45971:140704487245376][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpo1m3sdin
└── custom_validator

2 directories, 0 files


In [14]:
validator_demo.callbacks=dict(type='LogCallback', interval=5)

In [15]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-08-03 16:16:33,129 45971:140704487245376][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:33,132 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.006384 loss=10.000
[2023-08-03 16:16:33,134 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] ETA 0:00:00.003968 loss=20.000
[2023-08-03 16:16:33,135 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] ETA 0:00:00.001882 loss=30.000
[2023-08-03 16:16:33,137 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] ETA 0:00:00 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpzk9x8eqd
└── custom_validator

2 directories, 0 files


## Trainers

In [16]:
trainer_demo = validator_demo.copy()
trainer_demo.pop('type')
trainer_demo.dataloader = todd.Config(
    batch_size=2,
    shuffle=True,
    dataset=dict(type='RunnerDataset', n=67),
)
trainer_demo.optimizer = todd.Config(type='SGD', lr=0.005)


### Iteration Based

In [17]:
iter_based_trainer_demo = trainer_demo.copy()
iter_based_trainer_demo.type = 'CustomIterBasedTrainer'
iter_based_trainer_demo.name = 'custom_iter_based_trainer'
iter_based_trainer_demo.iters = 53

In [18]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        iter_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-03 16:16:33,432 45971:140704487245376][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:33,435 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.022387 loss=68.000 weight=0.000 batch={'x': tensor([66,  2]), 'y': tensor([132,   4])}
[2023-08-03 16:16:33,438 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.020752 loss=73.000 weight=0.000 batch={'x': tensor([52, 21]), 'y': tensor([104,  42])}
[2023-08-03 16:16:33,440 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] ETA 0:00:00.017726 loss=77.000 weight=0.000 batch={'x': tensor([44, 33]), 'y': tensor([88, 66])}
[2023-08-03 16:16:33,442 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.cust

### Epoch Based

In [19]:
epoch_based_trainer_demo = trainer_demo.copy()
epoch_based_trainer_demo.type = 'CustomEpochBasedTrainer'
epoch_based_trainer_demo.name = 'custom_epoch_based_trainer'
epoch_based_trainer_demo.epochs = 3

In [20]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        epoch_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-03 16:16:33,469 45971:140704487245376][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:33,470 45971:140704487245376][log.py:89 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-03 16:16:33,473 45971:140704487245376][log.py:83 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.052225 loss=92.000 weight=0.000 batch={'x': tensor([64, 28]), 'y': tensor([128,  56])}
[2023-08-03 16:16:33,475 45971:140704487245376][log.py:83 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] ETA 0:00:00.042614 loss=40.000 weight=0.000 batch={'x': tensor([34,  6]), 'y': tensor([68, 12])}
[2023-08-03 16:16:33,477 45971:140704487245376][log.py:83 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] ETA 0:00:00.037781 loss=109.

## Callbacks

### Log

In [21]:
log_callback_demo = validator_demo.copy()
log_callback = log_callback_demo.callbacks
log_callback.collect_env = todd.Config(verbose=False)
log_callback.with_file_handler = True
log_callback_demo.callbacks = [log_callback]

In [22]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        log_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-08-03 16:16:33,648 45971:140704487245376][log.py:50 todd.CustomValidator.custom_validator init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:33,653 45971:140704487245376][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:33,658 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.011835 loss=10.000
[2023-08-03 16:16:33,661 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] ETA 0:00:00.006097 loss=20.000
[2023-08-03 16:16:33,663 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp609c9fi8
└── custom_validator
    └── 2023-08-03T16-16-33_531839-08-00.log

2 directories, 1 file

[2023-08-03 16:16:33,648 45971:140704487245376][log.py:50 todd.CustomValidator.custom_validator init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:33,653 45971:140704487245376][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:33,658 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.011835 loss=10.000
[2023-08-03 16:16:33,661 45971:140704487245376][log.py:83 todd.CustomValidator.custom_validator after_run

### Optimize

In [23]:
optimize_callback_demo = iter_based_trainer_demo.copy()
optimize_callback = todd.Config(type='OptimizeCallback')
optimize_callback_demo.callbacks = [optimize_callback, log_callback]

In [24]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        optimize_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-03 16:16:34,324 45971:140704487245376][log.py:50 todd.CustomIterBasedTrainer.custom_iter_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:34,326 45971:140704487245376][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:34,332 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.038784 loss=25.113 weight=0.775 batch={'x': tensor([29, 12]), 'y': tensor([58, 24])}
[2023-08-03 16:16:34,335 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/

### Learning Rate Schedule

In [25]:
lr_schedule_callback_demo = iter_based_trainer_demo.copy()
lr_schedule_callback = todd.Config(
    type='LRScheduleCallback',
    lr_scheduler=dict(type='LinearLR', total_iters=10),
)
lr_schedule_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_callback,
    log_callback,
]


In [26]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-03 16:16:34,465 45971:140704487245376][log.py:50 todd.CustomIterBasedTrainer.custom_iter_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:34,467 45971:140704487245376][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:34,478 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.087533 loss=60.542 weight=0.245 batch={'x': tensor([36, 33]), 'y': tensor([72, 66])} lr=['3.333e-03']
[2023-08-03 16:16:34,513 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter

In [27]:
lr_schedule_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
lr_schedule_by_epoch_callback = lr_schedule_callback.copy()
lr_schedule_by_epoch_callback.by_epoch = True
lr_schedule_by_epoch_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_by_epoch_callback,
    log_callback,
]


In [28]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_by_epoch_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-03 16:16:34,676 45971:140704487245376][log.py:50 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:34,678 45971:140704487245376][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:34,680 45971:140704487245376][log.py:89 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-03 16:16:34,685 45971:140704487245376][log.py:83 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.101229 loss=99.167 weight=0.229 batch={'x': tensor([54, 58]), 'y': tens

### Learning Rate Scaler

In [29]:
lr_scaler_callback_demo = iter_based_trainer_demo.copy()
lr_scaler_callback = todd.Config(
    type='LRScaleCallback',
    lr_scaler=dict(base_batch_size=1),
)
lr_scaler_callback_demo.callbacks = [
    optimize_callback,
    lr_scaler_callback,
    log_callback,
]

In [30]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_scaler_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-03 16:16:34,806 45971:140704487245376][lr.py:92 todd.CustomIterBasedTrainer.custom_iter_based_trainer _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2023-08-03 16:16:34,918 45971:140704487245376][log.py:50 todd.CustomIterBasedTrainer.custom_iter_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:34,920 45971:140704487245376][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:34,927 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.032832 loss=48.620 weight=0.895 batch={'x': tensor([2

### Checkpoint

In [31]:
checkpoint_callback_demo = iter_based_trainer_demo.copy()
checkpoint_callback = todd.Config(type='CheckpointCallback', interval=10)
checkpoint_callback_demo.callbacks = [checkpoint_callback, log_callback]

In [32]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_50 = pathlib.Path(work_dirs) / 'custom_iter_based_trainer' / 'checkpoints' / 'iter_50'
    for f in iter_50.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-08-03 16:16:35,141 45971:140704487245376][log.py:50 todd.CustomIterBasedTrainer.custom_iter_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:35,143 45971:140704487245376][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:35,147 45971:140704487245376][log.py:83 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.016829 loss=68.000 weight=0.000 batch={'x': tensor([17, 51]), 'y': tensor([ 34, 102])}
[2023-08-03 16:16:35,151 45971:140704487245376][checkpoint.py:61 todd.CustomIterBasedTrainer.custom_iter_based_trainer _save] INFO: Saving st


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmptgrxdjgh
└── custom_iter_based_trainer
    ├── 2023-08-03T16-16-35_023548-08-00.log
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_20
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_30
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_40
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_50
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80

In [33]:
checkpoint_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
checkpoint_by_epoch_callback = checkpoint_callback.copy()
checkpoint_by_epoch_callback.update(interval=1, by_epoch=True)
checkpoint_by_epoch_callback_demo.callbacks = [
    checkpoint_by_epoch_callback,
    log_callback,
]

In [34]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_by_epoch_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'custom_epoch_based_trainer' / 'checkpoints' / 'epoch_2'
    for f in epoch_2.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-08-03 16:16:35,737 45971:140704487245376][log.py:50 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:35,739 45971:140704487245376][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:35,741 45971:140704487245376][log.py:89 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-03 16:16:35,744 45971:140704487245376][log.py:83 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.077076 loss=44.000 weight=0.000 batch={'x': tensor([15, 29]), 'y': tens


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp53_sjt06
└── custom_epoch_based_trainer
    ├── 2023-08-03T16-16-35_597787-08-00.log
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp53_sjt06/custom_epoch_based_trainer/checkpoints/epoch_3

7 directories, 16 files

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
    

In [35]:
checkpoint_load_from_callback_demo = checkpoint_by_epoch_callback_demo.copy()
checkpoint_load_from_callback_demo.callbacks = [
    optimize_callback,
    checkpoint_by_epoch_callback,
    log_callback,
]

In [36]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
        load_from=os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2')
    )
    runner.run()

[2023-08-03 16:16:36,332 45971:140704487245376][log.py:50 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:36,333 45971:140704487245376][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:36,335 45971:140704487245376][log.py:89 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-03 16:16:36,342 45971:140704487245376][log.py:83 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.138167 loss=49.853 weight=0.555 batch={'x': tensor([10, 59]), 'y': tens


--------------------



[2023-08-03 16:16:36,841 45971:140704487245376][checkpoint.py:46 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpjixqamyw/custom_epoch_based_trainer/checkpoints/epoch_2
[2023-08-03 16:16:36,922 45971:140704487245376][log.py:50 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:36,924 45971:140704487245376][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:36,925 45971:140704487245376][log.py:89 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epo

### Monitor

In [37]:
class CustomError(RuntimeError):
    pass

In [38]:
class FaultyRunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError('faulty runner')

In [39]:
@todd.RunnerRegistry.register()
class FaultyValidator(FaultyRunnerMixin, todd.runners.Validator):
    pass

In [40]:
@todd.RunnerRegistry.register()
class FaultyIterBasedTrainer(FaultyRunnerMixin, todd.runners.IterBasedTrainer):
    pass

In [41]:
@todd.RunnerRegistry.register()
class FaultyEpochBasedTrainer(
    FaultyRunnerMixin,
    todd.runners.EpochBasedTrainer,
):
    pass

In [42]:
monitor_callback_demo = validator_demo.copy()
monitor_callback_demo.type = 'FaultyValidator'
monitor_callback = todd.Config(type='MonitorCallback')
monitor_callback_demo.callbacks = [monitor_callback, log_callback]

In [43]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        monitor_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-08-03 16:16:37,105 45971:140704487245376][log.py:50 todd.FaultyValidator.custom_validator init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:37,106 45971:140704487245376][base.py:53 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:37,110 45971:140704487245376][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x153736a90>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", l

[2023-08-03 16:16:37,105 45971:140704487245376][log.py:50 todd.FaultyValidator.custom_validator init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:37,106 45971:140704487245376][base.py:53 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:37,110 45971:140704487245376][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x153736a90>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", l

### Priorities

## Strategies

In [44]:
strategy_load_model_from_demo = checkpoint_load_from_callback_demo.copy()

In [45]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.load_model_from(os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2', 'model.pth'))
    runner.run()

[2023-08-03 16:16:37,490 45971:140704487245376][log.py:50 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:37,492 45971:140704487245376][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:37,494 45971:140704487245376][log.py:89 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-03 16:16:37,499 45971:140704487245376][log.py:83 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.099367 loss=30.300 weight=0.738 batch={'x': tensor([46,  2]), 'y': tens


--------------------



[2023-08-03 16:16:38,067 45971:140704487245376][log.py:50 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 3997d99
Git status: M todd/runners/callbacks/checkpoint.py
[2023-08-03 16:16:38,069 45971:140704487245376][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-03 16:16:38,070 45971:140704487245376][base.py:60 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp7qhgjhft/custom_epoch_based_trainer/checkpoints/epoch_2/model.pth
[2023-08-03 16:16:38,073 45971:140704487245376][log.py:89 todd.CustomEpochBasedTrainer.custom_epoch_based_t

## Dry Run

In [46]:
todd.Store.DRY_RUN = True